<a href="https://colab.research.google.com/github/deckel28/music-mood/blob/master/mood_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import pickle
import h5py
import numpy as np
from numpy import savetxt
from numpy import genfromtxt
from numpy import random
from random import choices
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras
from keras import metrics
from keras import backend
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dropout, BatchNormalization
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from sklearn.metrics import multilabel_confusion_matrix

Using TensorFlow backend.


In [ ]:
os.chdir('/content/drive/My Drive')
os.listdir()

In [ ]:
trainY = genfromtxt('Datasets/encoded_train.csv', delimiter=',')
h5f = h5py.File('Datasets/feats_train.h5','r')
trainX = h5f['dataset'][:]
h5f.close()

valY = genfromtxt('Datasets/encoded_val.csv', delimiter=',')
h5f = h5py.File('Datasets/feats_val.h5','r')
valX = h5f['dataset'][:]
h5f.close()

testY = genfromtxt('Datasets/encoded_test.csv', delimiter=',')
h5f = h5py.File('Datasets/feats_test.h5','r')
testX = h5f['dataset'][:]
h5f.close()

In [ ]:
print('trainX-', trainX.shape, '   trainY- ', trainY.shape)
print('valX-', valX.shape, '      valY- ', valY.shape)
print('testX-', testX.shape, '     testY- ', testY.shape)

trainX- (2802, 20, 9762)    trainY-  (2802, 8)
valX- (346, 20, 9762)       valY-  (346, 8)
testX- (312, 20, 9762)      testY-  (312, 8)


In [ ]:
trainX0 = np.expand_dims(trainX, axis=-1)
valX0 = np.expand_dims(valX, axis=-1)
testX0 = np.expand_dims(testX, axis=-1)

IN_SHAPE = trainX0.shape[1:]
print(IN_SHAPE)

(20, 9762, 1)


In [ ]:
print(trainX0.shape, valX0.shape, testX0.shape)
print(trainY.shape, valY.shape, testY.shape)

(2802, 20, 9762, 1) (346, 20, 9762, 1) (312, 20, 9762, 1)
(2802, 8) (346, 8) (312, 8)


In [ ]:
def generator(features, labels, batch_size):
    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros((batch_size, IN_SHAPE[0], IN_SHAPE[1], 1))
    batch_labels = np.zeros((batch_size, 8))
    while True:
        for i in range(batch_size):
            index = random.choice(len(features), 1)
            batch_features[i] = features[index]
            batch_labels[i] = labels[index]
        yield batch_features, batch_labels

In [ ]:
def fbeta(y_true, y_pred, beta=2):
  # clip predictions
  y_pred = backend.clip(y_pred, 0, 1)
  # calculate elements
  tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
  fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
  fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
  # calculate precision
  p = tp / (tp + fp + backend.epsilon())
  # calculate recall
  r = tp / (tp + fn + backend.epsilon())
  # calculate fbeta, averaged across each class
  bb = beta ** 2
  fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
  return fbeta_score

In [ ]:
# train_yhat = np.asarray([np.ones(data_Y.shape[1]) for _ in range(data_Y.shape[0])])
# # test_yhat = np.asarray([np.ones(data_Y[180000:].shape[1]) for _ in range(data_Y[180000:].shape[0])])
# print(train_yhat.shape, data_Y.shape)
# # evaluate predictions with sklearn
# train_score = fbeta_score(data_Y, train_yhat, 2, average='samples')
# # test_score = fbeta_score(data_Y[180000:], test_yhat, 2, average='samples')
# # print('All Ones (sklearn): train=%.3f, test=%.3f' % (train_score, test_score))
# print(train_score)
# # evaluate predictions with keras
# train_score = fbeta(backend.variable(data_Y), backend.variable(train_yhat))
# # test_score = fbeta(backend.variable(data_Y[180000:]), backend.variable(test_yhat))
# # print('All Ones (keras): train=%.3f, test=%.3f' % (train_score, test_score))
# print(train_score)
# from sklearn.metrics import accuracy_score
# train_score = accuracy_score(data_Y, train_yhat)
# print(train_score)
# print(keras.metrics.accuracy(data_Y, train_yhat))

In [ ]:
def define_model(in_shape, out_shape):
  model = Sequential()
  
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=in_shape))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(60, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(out_shape, activation='sigmoid'))
  return model

In [ ]:
model = define_model(in_shape = IN_SHAPE, out_shape=8)
model.summary()
opt = keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy', 
               metrics=['accuracy', fbeta, keras.metrics.categorical_accuracy])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 20, 9762, 64)      640       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 20, 9762, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 10, 4881, 64)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 10, 4881, 64)      36928     
_________________________________________________________________
batch_normalization_4 (Batch (None, 10, 4881, 64)      256       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 10, 4881, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 2440, 64)      

In [ ]:
model.fit_generator(generator(trainX0, trainY, 24),
                    verbose=1,
                    steps_per_epoch=120,
                    epochs=10,
                    validation_data=(valX0, valY))

Epoch 1/10
120/120 [==============================] - 200s 2s/step - loss: 1.5792 - accuracy: 0.8122 - fbeta: 0.6224 - categorical_accuracy: 0.7069 - val_loss: 0.5963 - val_accuracy: 0.8306 - val_fbeta: 0.6747 - val_categorical_accuracy: 0.7832
Epoch 2/10
120/120 [==============================] - 192s 2s/step - loss: 0.3817 - accuracy: 0.8595 - fbeta: 0.6936 - categorical_accuracy: 0.7656 - val_loss: 0.3866 - val_accuracy: 0.8569 - val_fbeta: 0.6726 - val_categorical_accuracy: 0.8555
Epoch 3/10
120/120 [==============================] - 192s 2s/step - loss: 0.2904 - accuracy: 0.8840 - fbeta: 0.7402 - categorical_accuracy: 0.8163 - val_loss: 0.3717 - val_accuracy: 0.8551 - val_fbeta: 0.6648 - val_categorical_accuracy: 0.8121
Epoch 4/10
120/120 [==============================] - 192s 2s/step - loss: 0.2646 - accuracy: 0.9013 - fbeta: 0.7778 - categorical_accuracy: 0.8309 - val_loss: 0.3552 - val_accuracy: 0.8555 - val_fbeta: 0.6319 - val_categorical_accuracy: 0.7601
Epoch 5/10
120/120 [

In [ ]:
model.save('Models/model1.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


In [ ]:
thresholds = [['Happy', 0.8],
                ['Excited', 0.5],
                ['Frantic', 0.5],
                ['Anxious/Sad', 0.3],
                ['Anger', 0.5],
                ['Calm', 0.22],
                ['Tired', 0.5],
                ['Sensual', 0.1]]

In [ ]:
predictions = model.predict(trainX0)
y_pred=np.zeros((trainX0.shape[0],8))
for i in range(trainX0[:5000].shape[0]):
    for j in range(8):
        y_pred[i][j] = (predictions[i][j]>thresholds[j][1])
print(y_pred.shape)

(2802, 8)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss

accuracy = accuracy_score(trainY, y_pred)
print('Accuracy: %f' % accuracy)

f1 = fbeta(trainY, y_pred)
print('F1 score: %f' % f1)

hl = hamming_loss(trainY, y_pred)
print('Hamming Loss: %f' % hl)

Accuracy: 0.178087
F1 score: 0.798583
Hamming Loss: 0.134234
